---

<img src='common/logo_DH.png' align='left' width=35%/>

# <u>Workshop 2 - Grupo 2</u>

## DS Consultores - _INTEGRANTES:_
> ####      • CANGUEIRO, María Mercedes
> ####      • CHAMUT, Diego
> ####      • FERNÁNDEZ, Andrea
> ####      • GIORGETTI, María Gimena
> ####      • LLANOS, Santiago Francisco
> ####      • RIVAS NIETO, Christian Javier

## Modelos de ML para predecir el Precio Total en usd de una Propiedad


---
<img src="https://www.nocnok.com/hs-fs/hubfs/Documentos.jpg?width=1254&name=Documentos.jpg"
align="center"  width=50%/></img>

<a id="section_TOC"></a>
## Tabla de Contenidos

[1. Introducción](#section_intro)

[2. Dataset](#section_dataset)

$\hspace{.5cm}$[2.1. Importación de Datos](#section_2.1)

$\hspace{.5cm}$[2.2. Verificación de valores Nulos](#section_2.2)

$\hspace{.5cm}$[2.3. Imputación de valores Nulos](#section_2.3)

$\hspace{.5cm}$[2.4. Limpieza de Outliers](#section_2.4)

[3. Feature Engineering (Modelado)](#section_modelado)

[4. Regresión Lineal Simple](#section_lineal_simple)

$\hspace{.5cm}$[4.1. Regresión Lineal Simple Superficie Total](#section_4.1)

$\hspace{.5cm}$[4.2. Regresión Lineal Simple Ambientes](#section_4.2)

$\hspace{.5cm}$[4.3. Regresión Lineal Simple Estrenar](#section_4.3)

$\hspace{.5cm}$[4.4. Regresión Lineal Simple Cochera](#section_4.4)

$\hspace{.5cm}$[4.5. Regresión Lineal Simple Superficie Cubierta](#section_4.5)

[5. Regresión Lineal Multiple](#section_lineal_multiple)

$\hspace{.5cm}$[5.1. Regresión Lineal Multiple SIN Regularizacion](#section_5.1)

$\hspace{.5cm}$[5.2. Regresión Lineal Multiple CON Regularizacion](#section_5.2)

[6. Regresión Lineal Ridge](#section_lineal_ridge)
  
[7. Regresión Lineal Lasso](#section_lineal_lasso)

[8. Regresión Elastic Net](#section_elastic_net)

[9. Conclusiones ](#section_conclusiones)

$\hspace{.5cm}$[9.1. Comparación Numérica de los Distintos Modelos](#section_9.1)

$\hspace{.5cm}$[9.2. Comparación Gráfica de los Distintos Modelos](#section_9.2)

<a id="section_intro"></a>
## 1. Introducción

[volver a TOC](#section_TOC)


En este segundo workshop del curso de Data Science de Digital House, nos enfocaremos en desarrollar un modelo de regresión que permita predecir el precio total en USD de una propiedad, para que, en un futuro, la inmobiliaria Properati pueda utilizarlo como tasador automático en las próximas propiedades que sean comercializadas.
Realizaremos distintos modelos para poder comparar los resultados obtenidos y arribar al óptimo.<br>
Se plantearán diversas estrategias para abordar fallas y/o faltantes de información que perjudiquen el resultado del modelo, en vistas de lograr el objetivo mencionado anteriormente. <br>
Todo esto lo haremos mediante la aplicación de los conocimientos adquiridos hasta el momento a lo largo del cursado.

A continuación, efectuamos la **importación** de aquellas **librerías** que utilizaremos a lo largo de todo el trabajo.

In [ ]:
#Importamos pandas, numpy
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score,KFold
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

<a id="section_dataset"></a> 
## 2. Dataset

[volver a TOC](#section_TOC)

https://drive.google.com/file/d/0BzVrTKc02N8qNUdDSExBQlFTNlU/view?resourcekey=0-4m-28Uq6kWRDBrt2NXFbNQ

El dataset contiene información sobre todas las propiedades georeferenciadas de la base de datos de la empresa. 
La información que cada propiedad incluye es la siguiente:

**operation**: sell, rent, **property_type**: house, apartment, ph, **place_name**, **place_with_parent_names**, **country_name**, **state_name**, **geonames_id** (si está disponible), **lat-lon**, **price** (precio original del aviso), **currency**: ARS, USD, **price_aprox_local_currency**: ARS, **price_aprox_usd**, **surface_total_in_m2**, **surface_covered_in_m2**, **price_usd_per_m2**, **price_per_m2**, **floor**: (si corresponde), **rooms, expenses, properati_url, description, title, image_thumbnail**

Trabajamos con el dataset limpio obtenido luego de todo el proceso de ETL realizado en el Workshop 1. <br>

Adcionalmente, sólo nos enfocaremos en los registros de CABA y GBA (excluyendo los outliers), debido a que estas regiones contienen más del 80% de los registros del dataset original.

Realizaremos en esta etapa los pasos necesarios para: <br>
* Agregar las columnas que consideramos relevantes para nuestro análisis
* Quitar/imputar valores nulos que perjudiquen el modelo 
* Quitar outliers que no hayan sido limpiados en el trabajo anterior, utilizando nuevos métodos (como DBScan)
* Definir el dataset más ordenado posible, para luego entrenar nuestros modelos

<a id="section_2.1"></a> 
### 2.1. Importación de Datos

[volver a TOC](#section_TOC)

Importamos el dataset limpio, que resultó del trabajo integrador anterior:

In [ ]:
df_properatti = pd.read_csv('data/properatti_workshop2.csv')
df_properatti.head()

#### Creación de un df que contenga información georeferencial (GeoPandas):

In [ ]:
#Creamos df_prop que lo usaremos más adelante para crear algunos mapas:
df_prop = df_properatti[['place_name', 'lat', 'lon']]

In [ ]:
#Trabajamos en otro df que creamos sin valores nulos en la columna 'lat':
data_lat_notnull = df_prop.loc[df_prop.lat.notnull()]
data_lat_notnull.isnull().sum()

In [ ]:
#Calculamos la mediana de las latitudes por place_name:
data_coordenadas = data_lat_notnull.groupby('place_name')[['lat','lon']].median()
data_coordenadas.reset_index(inplace = True)
data_coordenadas.columns = ('place_name','lat_median','lon_median')
data_coordenadas.head(2)

In [ ]:
#Agregamos nuevas columnas con las medianas de lat y lon por place_name:
df_prop = pd.merge(df_prop, data_coordenadas, on='place_name', how = 'left')
df_prop.head(3)

In [ ]:
#Rellenamos valores nulos de lon y lat del df mayor, con las medianas:
df_prop.lat.fillna(df_prop.lat_median, inplace = True)

df_prop.lon.fillna(df_prop.lon_median, inplace = True)

In [ ]:
#Los datos que quedan vacíos es por que no hay ningún dato de geoposicionamiento en los datos del df:
df_prop.loc[df_prop.lat.isnull()].head(3) 

In [ ]:
#Eliminamos los valores que no tienen datos de latitud, que son los mismos que no tienen longitud:
df_prop.dropna(inplace = True, subset =['lat_median'])

In [ ]:
#Transformamos el df en geodf
geo_properatti = gpd.GeoDataFrame(df_prop,geometry=gpd.points_from_xy(df_prop.lon, df_prop.lat))
geo_properatti.head(3)

In [ ]:
#Agregamos un archivo shp para poder ver los departamentos de Argentina en forma de poligonos
departamentos_arg = gpd.read_file('data/departamento.shp')

In [ ]:
departamentos_arg.head(3)

#### Creación de Diccionario para generar nueva columna:

El objetivo es filtrar luego sólo las propiedades que pertenezcan a CABA y GBA.

In [ ]:
CABA_or_GBA_dict = {
    'Capital Federal'             : 'CABA',
    'Bs.As. G.B.A. Zona Sur'      : 'GBA',
    'Buenos Aires Costa Atlántica': 'Buenos Aires',
    'Bs.As. G.B.A. Zona Norte'    : 'GBA',
    'Bs.As. G.B.A. Zona Oeste'    : 'GBA',
    'Buenos Aires Interior'       : 'Buenos Aires'
}
#Creamos una columna nueva llamada 'CABA/GBA' con los valores que tiene la columna 'state_name'. Luego reemplazamos los valores de la columna 'CABA/GBA' por los del Diccionario:
df_properatti['CABA_or_GBA']= df_properatti['state_name']
df_properatti.replace({'CABA_or_GBA':CABA_or_GBA_dict}, inplace=True)

Chequeamos que efectivamente se haya creado la columna 'CABA/GBA' en el dataframe:

In [ ]:
df_properatti.head(3)

#### Selección de columnas relevantes para el análisis

Nos quedamos sólo con algunas columnas del Dataframe, que consideramos van a ser de utilidad luego en los modelos de ML.

In [ ]:
df_properatti=df_properatti[['property_type','CABA_or_GBA','place_name','balcon','terraza_patio','estrenar','cochera','ambientes','floor','ARS_to_USD_corregido','price_usd_per_m2','surface_total_in_m2', 'surface_covered_in_m2']]

df_properatti.shape

<a id="section_2.2"></a> 
### 2.2. Verificación de Valores Nulos

[volver a TOC](#section_TOC)

Vamos a corroborar si el Dataframe tiene valores nulos o Nulls (NaN) en alguna de sus columnas. De ser así, los registros que tengan Nulls habrá que eliminarlos o imputarlos de alguna forma, para que no perjudiquen a los modelos que utilizaremos luego.

In [ ]:
df_properatti.isnull().sum()

Como vemos, las columnas _estrenar_, _cochera_, _ambientes_, _floor_ y _surface_covered_in_m2_ son las únicas que tiene valores Nulls (NaN). Sin embargo, las columnas _estrenar_ y _cochera_ son binarias: 1 si ES "a estrenar" o si TIENE "cochera", y 0 en caso contrario. Por cómo fueron creadas, los Nulls en las mismas en realidad deberían ser ceros. <br>
Por lo tanto, procedemos a efectuar dicho reemplazo.

In [ ]:
df_properatti.estrenar.fillna(value=0 , inplace = True)
df_properatti.cochera.fillna(value=0 , inplace = True)

In [ ]:
df_properatti.isnull().sum()

La columna _floor_ vamos a descartarla, por tratarse en su mayoría de registros nulos (aprox. 93%), y considerar que no aporta información tan relevante como predictora. <br>
Por el contrario, vamos a enfocarnos en completar los datos de las columnas **superficie cubierta** (_surface_covered_in_m2_) y **ambientes** (_ambientes_), ya que consideramos que estas sí pueden ser de mayor utilidad como variables predictoras.    

<a id="section_2.3"></a> 
### 2.3. Imputación de Valores Nulos

[volver a TOC](#section_TOC)

#### Imputación de valores columnas _ambientes_ y _surface_covered_in_m2_

Vamos a imputar los valores faltantes en la columna _ambientes_ utilizando información de la columna _surface_covered_in_m2_. Para ello, primero tendremos que completar esta última, utilizando información de la columna _surface_total_in_m2_.<br>
Veamos primero la columna **_ambientes_.**

In [ ]:
df_amb = round(df_properatti.groupby('property_type')['ambientes'].describe(),2)
print('                     AMBIENTES')
df_amb

In [ ]:
pd.concat([df_amb.iloc[:,[1,5]], ((round(df_amb.iloc[:,1]/df_amb.iloc[:,5],2)-1)*100).rename('mean/median[%]')], axis=1)

Vemos en la tabla anterior datos "curiosos", probablemente erróneos, como _departamentos_ con _85 ambientes_. <br>

Sin embargo, por lo pronto, verificamos que para todos los tipos de propiedad, la mediana se encuentra relativamente cerca de la media, ligeramente por debajo de la misma. La diferencia promedio entre estas dos magnitudes, para todos los tipos de propiedad, es de aprox. 15%. Esta diferencia existe, seguramente, debido a la presencia de outliers, aunque probablemente no sean tantos, ya que los valores no son tan distantes entre sí. <br>

Tomaremos la **mediana** como valor de referencia a la hora de imputar, ya que además se trata de un número entero. <br>

Veamos ahora la columna **superficie cubierta** (_surface_covered_in_m2_).

In [ ]:
df_sup_c = round(df_properatti.groupby('property_type')['surface_covered_in_m2'].describe(),2)
print('                   SUPERFICIE CUBIERTA')
df_sup_c

In [ ]:
pd.concat([df_sup_c.iloc[:,[1,5]], ((round(df_sup_c.iloc[:,1]/df_sup_c.iloc[:,5],2)-1)*100).rename('mean/median[%]')], axis=1)

En este caso, la diferencia entre los dos estadísticos es mayor, lo que indica mayor presencia de outliers (o valores más extremos), sobre todo para el tipo de propiedad **stores**. Vemos que, en promedio, en los primeros 3 tipos de propiedad, hay una diferencia de aprox. 24% (la media por encima de la mediana), pero en el caso de stores, esta diferencia asciende a 171%, elevando el promedio total significativamente. <br>
Tendremos esto en cuenta a la hora de tratar de corregir posibles outliers y luego efectuar la imputación. 

In [ ]:
#Chequeamos cantidad de nulos por tipo de propiedad:
amb_nulls = df_properatti.ambientes.isnull().groupby(df_properatti['property_type']).sum()
sup_c_nulls = df_properatti.surface_covered_in_m2.isnull().groupby(df_properatti['property_type']).sum()
print('    CANT. DE NULOS POR TIPO DE PROPIEDAD')
pd.concat([amb_nulls, sup_c_nulls], axis=1)

Vemos que, tanto los **departamentos** como las **casas**, registran el mayor número de nulos para las dos columnas. 

In [ ]:
#Guardamos variables 
amb_nulls_PH = df_properatti.loc[(df_properatti['property_type']=='PH') & (df_properatti['ambientes'].isnull())]['ambientes']
amb_nulls_apart = df_properatti.loc[(df_properatti['property_type']=='apartment') & (df_properatti['ambientes'].isnull())]['ambientes']
amb_nulls_house = df_properatti.loc[(df_properatti['property_type']=='house') & (df_properatti['ambientes'].isnull())]['ambientes']
amb_nulls_store = df_properatti.loc[(df_properatti['property_type']=='store') & (df_properatti['ambientes'].isnull())]['ambientes']

Debemos averiguar los valores que vamos a utilizar para imputar. Primero, debemos completar la columna _surface_covered_in_m2_. Vamos a imputar datos desde _surface_total_in_m2_.<br> 
Averiguamos qué relación hay entre estas dos columnas, para cada tipo de propiedad.

In [ ]:
df_sup_summary = round(df_properatti.groupby('property_type')[['surface_covered_in_m2', 'surface_total_in_m2']].mean(),2)
df_sup_summary
print('     RELACIÓN ENTRE SUP. CUB. Y SUP. TOTAL POR TIPO DE PROPIEDAD')
df_sup_summ = pd.concat([df_sup_summary, ((round(df_sup_summary.iloc[:,0]/df_sup_summary.iloc[:,1],2))*100).rename('surf. cov./surf. total[%]')], axis=1)
df_sup_summ

Usaremos los valores de la **tercera columna** para imputar la sup. cubierta de cada tipo de propiedad.

In [ ]:
#Imputamos los nulos en SUPERFICIE CUBIERTA de cada tipo de propiedad:
#PH:
df_properatti.loc[(df_properatti['property_type']=='PH')&(df_properatti['surface_covered_in_m2'].isnull()), 'surface_covered_in_m2'] = \
                                                    df_properatti.surface_total_in_m2*(df_sup_summ.loc['PH','surf. cov./surf. total[%]']/100)
#Departamentos:
df_properatti.loc[(df_properatti['property_type']=='apartment')&(df_properatti['surface_covered_in_m2'].isnull()), 'surface_covered_in_m2'] = \
                                                    df_properatti.surface_total_in_m2*(df_sup_summ.loc['apartment','surf. cov./surf. total[%]']/100)

#House:
df_properatti.loc[(df_properatti['property_type']=='house')&(df_properatti['surface_covered_in_m2'].isnull()), 'surface_covered_in_m2'] = \
                                                    df_properatti.surface_total_in_m2*(df_sup_summ.loc['house','surf. cov./surf. total[%]']/100)

#Store:
df_properatti.loc[(df_properatti['property_type']=='store')&(df_properatti['surface_covered_in_m2'].isnull()), 'surface_covered_in_m2'] = \
                                                    df_properatti.surface_total_in_m2*(df_sup_summ.loc['store','surf. cov./surf. total[%]']/100)


In [ ]:
#Chequeamos nuevamente cantidad de nulos de superficie cubierta, por tipo de propiedad, verificando que sean todos cero:
amb_nulls = df_properatti.ambientes.isnull().groupby(df_properatti['property_type']).sum()
sup_c_nulls = df_properatti.surface_covered_in_m2.isnull().groupby(df_properatti['property_type']).sum()
print('    CANT. DE NULOS POR TIPO DE PROPIEDAD')
pd.concat([amb_nulls, sup_c_nulls], axis=1)

Con la tabla anterior, verificamos que, según lo que indica la segunda columna, ya no hay valores nulos para _surface_covered_in_m2_.<br>
Ahora vamos a hacer lo mismo para la columna **_ambientes_**.

In [ ]:
tabla_amb = df_properatti.groupby(['property_type', 'ambientes'])['surface_covered_in_m2'].describe().loc[:,['count','min','25%','50%','mean','75%','max']]
tabla_amb.columns=['count','min','25%','median(50%)','mean','75%','max']
tabla_amb

Podemos apreciar que hay inconsistencias en la progresión de la superficie cubierta en algunos casos. <br>
Para poder establecer intervalos, es necesario que aumente en "forma ascendente". En algunos casos, se puede ver que la mediana de superficie cubierta disminuye, para mayor número de ambientes. <br>
Esto no debería ocurrir, por lo que vamos a tratar de corregir esos datos.

In [ ]:
#Eliminamos valores extremos:
df_properatti.drop(df_properatti[(df_properatti.property_type=='apartment')&(df_properatti.ambientes>=12)].index, inplace = True)
df_properatti.drop(df_properatti[(df_properatti.property_type=='house')&(df_properatti.ambientes>=10)].index, inplace = True)
df_properatti.drop(df_properatti[(df_properatti.property_type=='store')&(df_properatti.ambientes>=4)].index, inplace = True)

In [ ]:
df_properatti.reset_index(inplace=True, drop = True)

In [ ]:
#Volvemos a chequear la tabla anterior:
tabla_amb = df_properatti.groupby(['property_type', 'ambientes'])['surface_covered_in_m2'].describe().loc[:,['count','min','25%','50%','mean','75%','max']]
tabla_amb.columns=['count','min','25%','median(50%)','mean','75%','max']
tabla_amb

In [ ]:
#Creamos las variables por tipo de propiedad para llamarlas luego en la imputación:
median_imput_factor = 1.1  
df_tabla_amb = tabla_amb.reset_index()
df_tabla_amb['lim_sup_interval_amb'] = df_tabla_amb['median(50%)'] * median_imput_factor

#PH:
int_amb_PH = list(df_tabla_amb['lim_sup_interval_amb'][df_tabla_amb.property_type == 'PH'])
int_amb_PH.insert(0,0)
nro_amb_PH = list(df_tabla_amb['ambientes'][df_tabla_amb.property_type == 'PH'])

#Departamentos:
int_amb_apartment = list(df_tabla_amb['lim_sup_interval_amb'][df_tabla_amb.property_type == 'apartment'])
int_amb_apartment.insert(0,0)
nro_amb_apartment = list(df_tabla_amb['ambientes'][df_tabla_amb.property_type == 'apartment'])

#Casas. No podemos utilizar los valores de la tabla anterior, porque debido a la inconsistencia en la progresión, se verifica que en algunos
#casos disminuye la superficie cubierta para un mayor nro. de ambientes. Por ese motivo, definimos el intervalo correspondiente:
int_amb_house = [60,90,120,160,210,260,310,390,450,520,50000]
int_amb_house.insert(0,0)
nro_amb_house = list(np.arange(1,12))

#Negocios:
int_amb_store = list(df_tabla_amb['lim_sup_interval_amb'][df_tabla_amb.property_type == 'store'])
int_amb_store.extend([300,400,20000])
int_amb_store.insert(0,0)
nro_amb_store = list(np.arange(1,7))

In [ ]:
#Guardamos variables 
sc_m2_nulls_PH = df_properatti.loc[(df_properatti['property_type']=='PH') & (df_properatti['ambientes'].isnull())]['surface_covered_in_m2']
sc_m2_nulls_apartment = df_properatti.loc[(df_properatti['property_type']=='apartment') & (df_properatti['ambientes'].isnull())]['surface_covered_in_m2']
sc_m2_nulls_house = df_properatti.loc[(df_properatti['property_type']=='house') & (df_properatti['ambientes'].isnull())]['surface_covered_in_m2']
sc_m2_nulls_store = df_properatti.loc[(df_properatti['property_type']=='store') & (df_properatti['ambientes'].isnull())]['surface_covered_in_m2']

In [ ]:
#Armamos los intervalos con la cantidad de ambientes correspondientes según la superficie cubierta: 
cant_amb_PH = pd.cut(sc_m2_nulls_PH, int_amb_PH, labels=nro_amb_PH)
cant_amb_apartment = pd.cut(sc_m2_nulls_apartment, int_amb_apartment, labels=nro_amb_apartment)
cant_amb_house = pd.cut(sc_m2_nulls_house, int_amb_house, labels=nro_amb_house)
cant_amb_store = pd.cut(sc_m2_nulls_store, int_amb_store, labels=nro_amb_store)

In [ ]:
#Creamos nuevas columnas para guardar estos datos y luego imputar llamando a estos valores:
df_properatti['cant_amb_PH'] = cant_amb_PH
df_properatti['cant_amb_apartment'] = cant_amb_apartment
df_properatti['cant_amb_house'] = cant_amb_house
df_properatti['cant_amb_store'] = cant_amb_store

In [ ]:
#Instanciamos la cantidad de categorías (cant. de ambientes) que podrá tomar dicha columna:
ambientes_categories = list(set(df_properatti.cant_amb_PH.cat.categories) | set(df_properatti.cant_amb_apartment.cat.categories) 
                            | set(df_properatti.cant_amb_house.cat.categories) | set(df_properatti.cant_amb_store.cat.categories))
ambientes_categories

In [ ]:
#Definimos la columna ambientes como categórica, para cargarle los intervalos posibles definidos anteriormente:
df_properatti.ambientes = pd.Categorical(df_properatti.ambientes, ordered=True)
df_properatti.ambientes = df_properatti.ambientes.cat.set_categories(ambientes_categories, ordered=True)
df_properatti.ambientes.dtypes

In [ ]:
#Imputamos los valores según el tipo de propiedad:
df_properatti.ambientes.fillna(df_properatti.cant_amb_PH, axis=0, inplace=True)
df_properatti.ambientes.fillna(df_properatti.cant_amb_apartment, axis=0, inplace=True)
df_properatti.ambientes.fillna(df_properatti.cant_amb_house, axis=0, inplace=True)
df_properatti.ambientes.fillna(df_properatti.cant_amb_store, axis=0, inplace=True)

In [ ]:
#Chequeamos nuevamente cantidad de nulos por tipo de propiedad, verificando que sean todos cero:
amb_nulls_2 = df_properatti.ambientes.isnull().groupby(df_properatti['property_type']).sum()
sup_c_nulls_2 = df_properatti.surface_covered_in_m2.isnull().groupby(df_properatti['property_type']).sum()
print('    CANT. DE NULOS POR TIPO DE PROPIEDAD')
pd.concat([amb_nulls_2, sup_c_nulls_2], axis=1)

Verificamos que ya no hay valores nulos tampoco en la columna _ambientes_. Hemos **completado este proceso de imputación**.<br>
Haremos la verificación final, antes de continuar a la siguiente etapa del análisis.

In [ ]:
#Revisamos la forma actual del DataFrame para revisar cantidad de filas y columnas:
df_properatti.shape

In [ ]:
df_properatti.sample(3)

In [ ]:
#Eliminamos las columnas creadas para este fin:
df_properatti = df_properatti.drop(['cant_amb_PH', 'cant_amb_apartment', 'cant_amb_house', 'cant_amb_store'], axis=1)
df_properatti.shape

In [ ]:
#Chequeamos las primeras filas para corroborar que las columnas se hayan eliminado correctamente:
df_properatti.head(5)

In [ ]:
df_properatti.info()

La columna o variable _ambientes_ la habíamos definido como "categórica", a los efectos de realizar el proceso previo. Ahora la vamos a convertir en variable de tipo "numérico", para utilizarla luego en los modelos de regresión. 

In [ ]:
df_properatti.ambientes = df_properatti.ambientes.astype(int)

In [ ]:
df_properatti.info()

Verificamos que figure como tipo _int32_.

<a id="section_2.4"></a> 
### 2.4. Limpieza de Outliers

[volver a TOC](#section_TOC)

Luego de imputar los valores vacíos de la columna _ambientes_ mediante su relación con la _superficie cubierta_, queremos controlar si existen otros valores poco representativos.<br>
Por lo que analizaremos si aún existen valores extremos para cada tipo de propiedad (apartment, house, PH, store), e intentaremos corregirlos, como último paso previo a la utilización de los modelos de regresión.

In [ ]:
df_properatti[df_properatti.property_type == 'apartment'].groupby(['property_type','surface_total_in_m2'])['ambientes'].mean() 
#un par de datos extraños en los extremos pero es el tipo de propiedad más completo para trabajar.

In [ ]:
df_properatti[df_properatti.property_type == 'house'].groupby(['property_type','surface_total_in_m2'])['ambientes'].mean() 
#muchos datos vacíos y extrañas relaciones de m2 y ambientes, perjudica al modelo que queremos trabajar.

In [ ]:
df_properatti[df_properatti.property_type == 'PH'].groupby(['property_type','surface_total_in_m2'])['ambientes'].mean()
#al igual que apartment, es un tipo de propiedad estable con sus datos, un poco de inconsistencias en los extremos pero correcto en general.

In [ ]:
df_properatti[df_properatti.property_type == 'store'].groupby(['property_type','surface_total_in_m2'])['ambientes'].mean() 
#no es tan representativo ya que 'store' no es común que posea ambientes, por lo que ensucia un poco el df en base a lo que queremos analizar.

## DBScan

Pudimos encontrar algunos valores poco representativos dentro de los datos, a estos los llamaremos outliers y buscamos eliminarlos para tener una moestra más exacta de los datos.
Para eliminarlos trabajaremos con el método no supervisado llamado DBSCAN, el cual crea clusters en función de los parametros eps y min_samples.
Esprilon (eps) representa el radio dentro del cual buscamos un cierto número de datos vecinos (min_samples) los cuales forman un cluster. Los valores que no tienen otros valores vecinos dentro del radio estipulado, se consideran outliers y el modelo los agrupa en el cluster numerado como -1.

In [ ]:
#Generamos un df para analizar los outliers

#aquí podemos setear las variables que queramos analizar, pero hay que modificar un par de códigos más abajo si agregamos la 3ra feature
feature_0 = 'place_name'
feature_1 = 'surface_covered_in_m2'
feature_2 = 'ambientes'
feature_3 = 'surface_total_in_m2'

features_dbscan = [feature_0,feature_1,feature_2]#,feature_3]

df_dbscan = df_properatti[features_dbscan]#[df_properatti.property_type == 'apartment'] podemos filtrar a su vez por tipo de propiedad
df_dbscan

In [ ]:
#Graficamos los datos para tener una idea de su distribución
plt.scatter (df_dbscan[feature_1],df_dbscan[feature_2], s=10)

In [ ]:
#Aplicamos DBSCAN para asociar los valores a los clusters y los outliers quedan separados en el cluster -1
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

X_db = StandardScaler().fit_transform(df_dbscan[[feature_1,feature_2]])#,feature_3]])
dbscan = DBSCAN(eps = 0.5, min_samples = 5)
labels = dbscan.fit_predict(X_db)
np.unique(labels)

In [ ]:
#Shape de outliers
df_dbscan[[feature_1,feature_2]][labels==-1].shape

In [ ]:
#Ploteamos los primeros 10 clusters
plt.scatter (df_dbscan[feature_1][labels==-1], df_dbscan[feature_2][labels==-1], s=10, c='black') #representan los outliers

plt.scatter (df_dbscan[feature_1][labels==0], df_dbscan[feature_2][labels==0], s=10, c='r')
plt.scatter (df_dbscan[feature_1][labels==1], df_dbscan[feature_2][labels==1], s=10, c='g')
plt.scatter (df_dbscan[feature_1][labels==2], df_dbscan[feature_2][labels==2], s=10, c='yellow')
plt.scatter (df_dbscan[feature_1][labels==3], df_dbscan[feature_2][labels==3], s=10, c='brown')
plt.scatter (df_dbscan[feature_1][labels==4], df_dbscan[feature_2][labels==4], s=10, c='grey')
plt.scatter (df_dbscan[feature_1][labels==5], df_dbscan[feature_2][labels==5], s=10, c='pink')
plt.scatter (df_dbscan[feature_1][labels==6], df_dbscan[feature_2][labels==6], s=10, c='purple')
plt.scatter (df_dbscan[feature_1][labels==7], df_dbscan[feature_2][labels==7], s=10, c='orange')
plt.scatter (df_dbscan[feature_1][labels==8], df_dbscan[feature_2][labels==8], s=10, c='lightblue')
plt.scatter (df_dbscan[feature_1][labels==9], df_dbscan[feature_2][labels==9], s=10, c='steelblue')
plt.scatter (df_dbscan[feature_1][labels==10], df_dbscan[feature_2][labels==10], s=10, c='lime')

plt.xlabel(feature_1)
plt.ylabel(feature_2)

plt.show()

In [ ]:
#Ploteamos los outliers para tener una clara idea de su ubicación
plt.scatter(df_dbscan[feature_1][labels==-1], df_dbscan[feature_2][labels==-1], s=10 , c='black')
plt.xlabel(feature_1)
plt.ylabel(feature_2)
plt.show()

In [ ]:
#Creamos df de outliers para dropear del df original
df_to_drop = df_dbscan[labels==-1]
df_to_drop.sample(5)

In [ ]:
#Añadimos los datos geoespaciales a los outliers para ver si existe alguna relación dentro del mapa
geo_properatti_oultiers = df_to_drop.merge(geo_properatti['geometry'], left_index = True, right_index=True, how = 'left')
geo_properatti_oultiers.head(3)

In [ ]:
#Transformamos el df de outliers en geodf
geo_properatti_oultiers = gpd.GeoDataFrame(geo_properatti_oultiers)

In [ ]:
#Plasmamos los outliers dentro del mapa de Buenos Aires dividido en departamentos
fig, ax = plt.subplots(figsize=(10, 10))
departamentos_arg.plot(ax=ax, color='white', edgecolor='black')


geo_properatti_oultiers.plot(color='red', ax=ax, zorder=5, markersize = 50, marker = 'x', label = 'outliers')

ax.set_title('Ubicación de los outliers', 
             pad = 20, 
             fontdict={'fontsize':20, 'color': '#4873ab'})

ax.set_xlabel('Longitud')
ax.set_ylabel('Latitud')

ax.set(xlim=(-59.1, -58.3), ylim=( -34.95, -34.25))

ax.legend()

fig.show()

Como puede verse, no existe relacion entre los outliers y su ubicación en el mapa

In [ ]:
#Eliminamos los outliers encontrados
df_properatti = df_properatti.drop(index = df_to_drop.index)
df_properatti = df_properatti.reset_index(drop = True)

<br>
Chequeamos si DBSCAN pudo detectar los datos no representativos, vemos que algunos errores desaparecieron

In [ ]:
df_properatti[df_properatti.property_type == 'apartment'].groupby(['property_type','surface_total_in_m2'])['ambientes'].mean() 
#un par de datos extraños en los extremos pero es el tipo de propiedad más completo para trabajar

In [ ]:
df_properatti[df_properatti.property_type == 'house'].groupby(['property_type','surface_total_in_m2'])['ambientes'].mean() 
#muchos datos vacíos y extrañas relaciones de m2 y ambientes, perjudica al modelo qeu qeuremos trabajar

In [ ]:
df_properatti[df_properatti.property_type == 'PH'].groupby(['property_type','surface_total_in_m2'])['ambientes'].mean()
#al igual que atartment, es un tipo de propiedad estable con sus datos, un poco de inconsistencias en los extremos pero correcto en general

In [ ]:
df_properatti[df_properatti.property_type == 'store'].groupby(['property_type','surface_total_in_m2'])['ambientes'].mean() 
#no es tan representativo ya que 'store' no es comun que posea ambientes, por lo que ensucia un poco el df en base a lo que queremos analizar

Vemos que, antes de eliminar los registros con NaN en la columna "ambientes" teníamos 43348 registros, y luego de eliminarlos tenemos 30576 registros en el Dataframe "df_properatti".<br>

#### Ahora veamos cómo se relacionan las columnas entre sí con la Matriz de Correlación:

In [ ]:
#Graficamos la matriz de correlacion:
fig, ax = plt.subplots(figsize=(9.5,9.5))       
sns.heatmap(df_properatti.corr() , square=True, annot=True, ax=ax);

In [ ]:
# Visualizamos la matriz de Correlación en Seaborn usando a heatmap:
sns.heatmap(df_properatti.corr() , vmin=-1, vmax=1, center= 0, cmap="YlGnBu");

Del gráfico anterior, observamos que hay una alta correlación entre el Precio Total en USD ('ARS_to_USD_corregido'), la Superficie Total ('surface_total_in_m2'), la Superficie Cubierta ('surface_covered_in_m2') y los Ambientes (ambientes), de modo que vamos a usar dichas features por separado para intentar predecir el Precio Total en USD con el modelo de Regresión Lineal Simple. Posteriormente combinaremos todas las features en una Regresión Lineal Multiple para ver si tenemos una mayor precicion a la hora de predecir el valor de una propiedad.

En este gráfico podemos ver la relacion lineal entre todas las variables del df

In [ ]:
#aquí podemos ver la relación 1 a 1 entre todas las variables del df
sns.pairplot(df_properatti);

In [ ]:
#Vamos a renombrar la columna llamada 'place_name' a 'barrio' para mayor comodidad:
df_properatti.rename(columns={'place_name':'barrio'}, inplace=True)

<a id="section_modelado"></a> 
## 3. Feature Engineering (Modelado)

[volver a TOC](#section_TOC)

Para comenzar el modelado, transformaremos las variables Categóricas en dummies, luego más adelante normalizaremos las variables para facilitar la corrida de los distintos modelos (Cuando veamos la Regresión Lineal Múltiple).

Vamos a crear variables Dummies para las siguientes variables Categóricas:
- property_type
- CABA_or_GBA
- barrio (anteriormente llamada 'place_name')


In [ ]:
#Creamos variables dummies de las variables categóricas:

property_type_dummies=pd.get_dummies(df_properatti.property_type, prefix= 'property_type', drop_first=True)

CABA_or_GBA_dummies=pd.get_dummies(df_properatti.CABA_or_GBA, drop_first=True) #quité el pefix porque era medio confuso CABA_or_GBA_GBA

barrio_dummies=pd.get_dummies(df_properatti.barrio, drop_first=True) #quite el prefix porque necesito los nombres de los barrios limpios

In [ ]:
#Concatenamos el Dataframe original y los Dummy Dataframes (axis=0 significa filas, axis=1 significa columnas):
df_properatti=pd.concat([df_properatti,property_type_dummies], axis=1)
df_properatti=pd.concat([df_properatti,CABA_or_GBA_dummies], axis=1)
df_properatti=pd.concat([df_properatti,barrio_dummies], axis=1)

### Chequeamos si se generaron correctamente las columnas de las variables Dummies en nuestro Dataframe original:

In [ ]:
df_properatti.head()

<a id="section_lineal_simple"></a> 
## 4. Regresión Lineal Simple

[volver a TOC](#section_TOC)

#### Importamos todas las librerías necesarias para la Regresión

In [ ]:
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn import metrics

Representamos gráficamente las relaciones lineales entre las features elegidas por su correlación con el precio total en USD (surface_total_in_m2, ambientes y surface_covered_in_m2), y observamos como son las pendientes según son de CABA o GBA, o según el tipo de propiedad y como útltima imagen vemos la regresión lineal en general

In [ ]:
#Hacemos el Scatterplot entre 'surface_total_in_m2' y 'ARS_to_USD' para ver a priori qué correlación hay:
sns.pairplot(df_properatti, x_vars=["surface_total_in_m2", "ambientes", "surface_covered_in_m2"], y_vars=["ARS_to_USD_corregido"],
              hue='CABA_or_GBA', height=5, aspect=.8, kind="reg").set(title = 'Distribucion por CABA o GBA');
sns.pairplot(df_properatti, x_vars=["surface_total_in_m2", "ambientes", "surface_covered_in_m2"], y_vars=["ARS_to_USD_corregido"],
              hue='property_type', height=5, aspect=.8, kind="reg").set(title = 'Distribucion por tipo de propiedad');
sns.pairplot(df_properatti, x_vars=["surface_total_in_m2", "ambientes", "surface_covered_in_m2"], y_vars=["ARS_to_USD_corregido"],
              height=5, aspect=.8, kind="reg").set(title = 'Distribucion en general');

Definimos una función y la llamamos 'train_test_error' que requiera como parámetro una lista de features (feature_cols), genere la matriz de variables independientes 'X' y el array de la variable Target 'y' para luego hacer el split entre train y test reservando un 25% de las observaciones para testeo, y que finalmente imprima los errores MAE, MSE, RMSE y R2:

In [ ]:
def train_test_error(feature_cols):
    
    #Armamos nuestran matriz de features (X) y nuestro vector objetivo (y):
    X = df_properatti[feature_cols]
    y = df_properatti.ARS_to_USD_corregido
    
    #Separamos los datos en sets de testeo y training:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
    
    #Instanciamos el modelo:
    linreg = linear_model.LinearRegression()
    
    #Entrenamos el modelo:
    reg_lineal = linreg.fit(X_train, y_train)
    
    #Calculamos el y_pred:
    y_pred = linreg.predict(X_test)
    
    #Guardamos en una variable al Score (R2) del modelo para poder comparar:
    R2_reg_lineal= reg_lineal.score(X_test, y_test)
    
    print (feature_cols)
    #Imprimimos los coeficientes:
    print('Intercept: ', linreg.intercept_.round(3))
    print('Coeficients: ',linreg.coef_.round(3))
    #para observarlo mejor miramos el nombre de la variable con el coeficiente:
    print(list(zip(feature_cols, linreg.coef_.round(3))))
    
    #Imprimimos todas las métricas que nos interesan para poder comparar:
    print ('y_test_sample:' , (y_test.values[0:5]))
    print ('y_pred_sample:', y_pred[0:5].astype(int))
    print ('MAE: ', metrics.mean_absolute_error(y_test, y_pred).round(3))
    print ('MSE: ', metrics.mean_squared_error(y_test, y_pred).round(3))
    print ('RMSE: ', np.sqrt(metrics.mean_squared_error(y_test, y_pred)).round(3))
    print ('R2_train: ', reg_lineal.score(X_train, y_train).round(3))
    print ('R2_test: ', reg_lineal.score(X_test, y_test).round(3))
    print(f'\n')
    return

Utilizando la función 'train_test_error' recién creada, probamos y comparamos diferentes ensambles de features:

In [ ]:
train_test_error(['surface_total_in_m2'])
train_test_error(['surface_covered_in_m2'])
train_test_error(['ambientes'])
train_test_error(['estrenar'])
train_test_error(['cochera'])

Como la función no almacena las variables creadas, procedemos a trabajar todas las variables (features) por separado para poder almacenar sus resultados y compararlos al final del trabajo y así ver cuál modelo resulto ser el más representaivo.
Agregamos al final de cada ejecución un gráfico que representa los valores predichos con el modelo vs los valores reales.

<a id="section_4.1"></a> 
### 4.1 Regresión Lineal Simple - _Superficie Total_

[volver a TOC](#section_TOC)

In [ ]:
#Armamos nuestran matriz de features (X) y nuestro vector objetivo (y):
feature_cols = ['surface_total_in_m2']
X = df_properatti[feature_cols]
y = df_properatti.ARS_to_USD_corregido
    
#Separamos los datos en sets de testeo y training:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
    
#Instanciamos el modelo:
linreg = linear_model.LinearRegression()
    
#Entrenamos el modelo:
reg_lineal_m2 = linreg.fit(X_train, y_train)
    
#Calculamos el y_pred:
y_pred = linreg.predict(X_test)
    
#Guardamos en una variable al Score (R2) del modelo para poder comparar:
R2_reg_lineal_m2= reg_lineal_m2.score(X_test, y_test)

print (feature_cols)    
#Imprimimos los coeficientes:
print('Intercept: ', linreg.intercept_.round(3))
print('Coeficients: ',linreg.coef_.round(3))
#para observarlo mejor miramos el nombre de la variable con el coeficiente:
print(list(zip(feature_cols, linreg.coef_.round(3))))
    
#Imprimimos todas las métricas que nos interesan para poder comparar:
print ('y_test_sample:' , y_test.values[0:10])
print ('y_pred_sample:', y_pred[0:10].astype(int))
print ('MAE: ', metrics.mean_absolute_error(y_test, y_pred).round(3))
print ('MSE: ', metrics.mean_squared_error(y_test, y_pred).round(3))
print ('RMSE: ', np.sqrt(metrics.mean_squared_error(y_test, y_pred)).round(3))
print ('R2_train: ', reg_lineal_m2.score(X_train, y_train).round(3))
print ('R2_test: ', reg_lineal_m2.score(X_test, y_test).round(3))
print()

In [ ]:
reg_lineal_simple_model_m2 = {'Modelo': "Regresion Lineal Simple M2",
                        'Alcance del modelo': 'Buenos Aires',
                        'Variable Objetivo': 'ARS_to_USD_corregido',
                        'Cantidad de Observaciones': len(df_properatti),
                        'R2_train': reg_lineal_m2.score(X_train, y_train),
                        'R2_test': R2_reg_lineal_m2,
                        'Intercepto': reg_lineal_m2.intercept_}

In [ ]:
plt.plot(y,y, '-.',c='grey')
plt.scatter(y_pred, y_test, s=30, c='r', marker='+', zorder=10)
plt.xlabel("Predicciones (y_pred)")
plt.ylabel("Valores reales (y_test)")
plt.title(feature_cols)
plt.show()

<a id="section_4.2"></a> 
### 4.2 Regresión Lineal Simple - _Ambientes_

[volver a TOC](#section_TOC)

In [ ]:
#Armamos nuestran matriz de features (X) y nuestro vector objetivo (y):
feature_cols = ['ambientes']
X = df_properatti[feature_cols]
y = df_properatti.ARS_to_USD_corregido
    
#Separamos los datos en sets de testeo y training:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
    
#Instanciamos el modelo:
linreg = linear_model.LinearRegression()
    
#Entrenamos el modelo:
reg_lineal_amb = linreg.fit(X_train, y_train)
    
#Calculamos el y_pred:
y_pred = linreg.predict(X_test)
    
#Guardamos en una variable al Score (R2) del modelo para poder comparar:
R2_reg_lineal_amb= reg_lineal_amb.score(X_test, y_test)

print (feature_cols)
#Imprimimos los coeficientes:
print('Intercept: ', linreg.intercept_.round(3))
print('Coeficients: ',linreg.coef_.round(3))
#para observarlo mejor miramos el nombre de la variable con el coeficiente:
print(list(zip(feature_cols, linreg.coef_.round(3))))
    
#Imprimimos todas las métricas que nos interesan para poder comparar:
print ('y_test_sample:' , y_test.values[0:10])
print ('y_pred_sample:', y_pred[0:10].astype(int))
print ('MAE: ', metrics.mean_absolute_error(y_test, y_pred).round(3))
print ('MSE: ', metrics.mean_squared_error(y_test, y_pred).round(3))
print ('RMSE: ', np.sqrt(metrics.mean_squared_error(y_test, y_pred)).round(3))
print ('R2_train: ', reg_lineal_amb.score(X_train, y_train).round(3))
print ('R2_test: ', reg_lineal_amb.score(X_test, y_test).round(3))
print()

In [ ]:
reg_lineal_simple_model_amb = {'Modelo': "Regresion Lineal Simple Ambientes",
                        'Alcance del modelo': 'Buenos Aires',
                        'Variable Objetivo': 'ARS_to_USD_corregido',
                        'Cantidad de Observaciones': len(df_properatti),
                        'R2_train': reg_lineal_amb.score(X_train, y_train),
                        'R2_test': R2_reg_lineal_amb,
                        'Intercepto': reg_lineal_amb.intercept_}

In [ ]:
# Graficamos el modelo
plt.scatter(y_pred, y_test, alpha = 0.5,s=30, c='r', marker='x', zorder=10)
plt.plot(y,y, '-.',c='blue')  # con esto graficamos la recta y=x , o sea que ambas variables x e y tomen el mismo valor
plt.xlabel("Predicciones (y_pred)")
plt.ylabel("Valores reales (y_test)")
plt.title(feature_cols)
plt.show()

<a id="section_4.3"></a> 
### 4.3 Regresión Lineal Simple - _Estrenar_

[volver a TOC](#section_TOC)

In [ ]:
#Armamos nuestran matriz de features (X) y nuestro vector objetivo (y):
feature_cols = ['estrenar']
X = df_properatti[feature_cols]
y = df_properatti.ARS_to_USD_corregido
    
#Separamos los datos en sets de testeo y training:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
    
#Instanciamos el modelo:
linreg = linear_model.LinearRegression()
    
#Entrenamos el modelo:
reg_lineal_est = linreg.fit(X_train, y_train)
    
#Calculamos el y_pred:
y_pred = linreg.predict(X_test)
    
#Guardamos en una variable al Score (R2) del modelo para poder comparar:
R2_reg_lineal_est= reg_lineal_est.score(X_test, y_test)

print(feature_cols)
#Imprimimos los coeficientes:
print('Intercept: ', linreg.intercept_.round(3))
print('Coeficients: ',linreg.coef_.round(3))
#para observarlo mejor miramos el nombre de la variable con el coeficiente:
print(list(zip(feature_cols, linreg.coef_.round(3))))
    
#Imprimimos todas las métricas que nos interesan para poder comparar:
print ('y_test_sample:' , y_test.values[0:10])
print ('y_pred_sample:', y_pred[0:10].astype(int))
print ('MAE: ', metrics.mean_absolute_error(y_test, y_pred).round(3))
print ('MSE: ', metrics.mean_squared_error(y_test, y_pred).round(3))
print ('RMSE: ', np.sqrt(metrics.mean_squared_error(y_test, y_pred)).round(3))
print ('R2_train: ', reg_lineal_est.score(X_train, y_train).round(3))
print ('R2_test: ', reg_lineal_est.score(X_test, y_test).round(3))
print()

In [ ]:
reg_lineal_simple_model_est = {'Modelo': "Regresion Lineal Simple Estrenar",
                        'Alcance del modelo': 'Buenos Aires',
                        'Variable Objetivo': 'ARS_to_USD_corregido',
                        'Cantidad de Observaciones': len(df_properatti),
                        'R2_train': reg_lineal_est.score(X_train, y_train),
                        'R2_test': R2_reg_lineal_est,
                        'Intercepto': reg_lineal_est.intercept_}

In [ ]:
# Graficamos el modelo
plt.scatter(y_pred, y_test, alpha = 0.5,s=30, c='r', marker='x', zorder=10)
plt.plot(y,y, '-.',c='blue')  # con esto graficamos la recta y=x , o sea que ambas variables x e y tomen el mismo valor
plt.xlabel("Predicciones (y_pred)")
plt.ylabel("Valores reales (y_test)")
plt.title(feature_cols)
plt.show()

<a id="section_4.4"></a> 
### 4.4 Regresión Lineal Simple - _Cochera_

[volver a TOC](#section_TOC)

In [ ]:
#Armamos nuestran matriz de features (X) y nuestro vector objetivo (y):
feature_cols = ['cochera']
X = df_properatti[feature_cols]
y = df_properatti.ARS_to_USD_corregido
    
#Separamos los datos en sets de testeo y training:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
    
#Instanciamos el modelo:
linreg = linear_model.LinearRegression()
    
#Entrenamos el modelo:
reg_lineal_coche = linreg.fit(X_train, y_train)
    
#Calculamos el y_pred:
y_pred = linreg.predict(X_test)
    
#Guardamos en una variable al Score (R2) del modelo para poder comparar:
R2_reg_lineal_coche= reg_lineal_coche.score(X_test, y_test)
    
print (feature_cols)
#Imprimimos los coeficientes:
print('Intercept: ', linreg.intercept_.round(3))
print('Coeficients: ',linreg.coef_.round(3))
#para observarlo mejor miramos el nombre de la variable con el coeficiente:
print(list(zip(feature_cols, linreg.coef_.round(3))))
    
#Imprimimos todas las métricas que nos interesan para poder comparar:
print ('y_test_sample:' , y_test.values[0:10])
print ('y_pred_sample:', y_pred[0:10].astype(int))
print ('MAE: ', metrics.mean_absolute_error(y_test, y_pred).round(3))
print ('MSE: ', metrics.mean_squared_error(y_test, y_pred).round(3))
print ('RMSE: ', np.sqrt(metrics.mean_squared_error(y_test, y_pred)).round(3))
print ('R2_train: ', reg_lineal_coche.score(X_train, y_train).round(3))
print ('R2_test: ', reg_lineal_coche.score(X_test, y_test).round(3))
print()

In [ ]:
reg_lineal_simple_model_cochera = {'Modelo': "Regresion Lineal Simple Cochera",
                        'Alcance del modelo': 'Buenos Aires',
                        'Variable Objetivo': 'ARS_to_USD_corregido',
                        'Cantidad de Observaciones': len(df_properatti),
                        'R2_train': reg_lineal_coche.score(X_train, y_train),
                        'R2_test': R2_reg_lineal_coche,
                        'Intercepto': reg_lineal_coche.intercept_}

In [ ]:
# Graficamos el modelo
plt.scatter(y_pred, y_test, alpha = 0.5,s=30, c='r', marker='x', zorder=10)
plt.plot(y,y, '-.',c='blue')  # con esto graficamos la recta y=x , o sea que ambas variables x e y tomen el mismo valor
plt.xlabel("Predicciones (y_pred)")
plt.ylabel("Valores reales (y_test)")
plt.title(feature_cols)
plt.show()

<a id="section_4.5"></a> 
### 4.5 Regresión Lineal Simple - _Superficie Cubierta_

[volver a TOC](#section_TOC)

In [ ]:
#Armamos nuestran matriz de features (X) y nuestro vector objetivo (y):
feature_cols = ['surface_covered_in_m2']
X = df_properatti[feature_cols]
y = df_properatti.ARS_to_USD_corregido
    
#Separamos los datos en sets de testeo y training:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
    
#Instanciamos el modelo:
linreg = linear_model.LinearRegression()
    
#Entrenamos el modelo:
reg_lineal_scm2 = linreg.fit(X_train, y_train)
    
#Calculamos el y_pred:
y_pred = linreg.predict(X_test)
    
#Guardamos en una variable al Score (R2) del modelo para poder comparar:
R2_reg_lineal_scm2= reg_lineal_scm2.score(X_test, y_test)
    
print (feature_cols)
#Imprimimos los coeficientes:
print('Intercept: ', linreg.intercept_.round(3))
print('Coeficients: ',linreg.coef_.round(3))
#para observarlo mejor miramos el nombre de la variable con el coeficiente:
print(list(zip(feature_cols, linreg.coef_.round(3))))
    
#Imprimimos todas las métricas que nos interesan para poder comparar:
print ('y_test_sample:' , y_test.values[0:10])
print ('y_pred_sample:', y_pred[0:10].astype(int))
print ('MAE: ', metrics.mean_absolute_error(y_test, y_pred).round(3))
print ('MSE: ', metrics.mean_squared_error(y_test, y_pred).round(3))
print ('RMSE: ', np.sqrt(metrics.mean_squared_error(y_test, y_pred)).round(3))
print ('R2_train: ', reg_lineal_scm2.score(X_train, y_train).round(3))
print ('R2_test: ', reg_lineal_scm2.score(X_test, y_test).round(3))
print()

In [ ]:
reg_lineal_simple_model_scm2 = {'Modelo': "Regresion Lineal Simple m2 cubiertos",
                        'Alcance del modelo': 'Buenos Aires',
                        'Variable Objetivo': 'ARS_to_USD_corregido',
                        'Cantidad de Observaciones': len(df_properatti),
                        'R2_train': reg_lineal_scm2.score(X_train, y_train),
                        'R2_test': R2_reg_lineal_scm2,
                        'Intercepto': reg_lineal_scm2.intercept_}

In [ ]:
# Graficamos el modelo
plt.scatter(y_pred, y_test, alpha = 0.5,s=30, c='r', marker='x', zorder=10)
plt.plot(y,y, '-.',c='blue')  # con esto graficamos la recta y=x , o sea que ambas variables x e y tomen el mismo valor
plt.xlabel("Predicciones (y_pred)")
plt.ylabel("Valores reales (y_test)")
plt.title(feature_cols)
plt.show()

#### Conclusión de la Regresión Lineal Simple aplicada a las features de manera individual
Vemos que es una dispersión que no se puede representar con una recta de una manera que tenga precisión, ya que cualquier recta que se trace, por más de  que minimice los errores, siempre va a haber muchos puntos por encima de la recta y muchos por debajo. Por lo que ya sabemos de antemano que este modelo de Regresión no va a lograr un nivel aceptable de predicción.

<a id="section_lineal_multiple"></a> 
## 5. Regresión Lineal Multiple

[volver a TOC](#section_TOC)

Podemos observar que el modelo de Regresión Lineal Simple es demasiado simple para poder obtener un insight lo suficientemente convincente, por lo cual vamos a aplicar un modelo con todas las features al mismo tiempo (Regrsión Lineal Multiple) y posteriormente otros dos modelos que incluyen técnicas de Regularizacion (Ridge y Lasso) para mejorar el R2 del modelo:

<a id="section_5.1"></a> 
## 5.1 Regresión Lineal Multiple _SIN Regularización_ 
(sin estandarizacion de variables numericas ni variables dummies para variables categoricas)

[volver a TOC](#section_TOC)

Empecemos realizando una Regresión Lineal Múltiple sin Estandarización de Features Numéricas ni Creación de variables Dummies, para ver cómo nos da el Modelo. 
Usaremos 5 variables predictoras en nuestro modelo de Regresión Lineal Múltiple para empezar: "surface_total_in_m2", "ambientes", "estrenar", "cochera" y "surface_covered_in_m2". 
Crearemos la serie "feature_cols" con esas variables, y luego la meteremos en la función "train_test_error" que habíamos creado para que nos evalúe el Modelo:

In [ ]:
train_test_error(['surface_total_in_m2','surface_covered_in_m2','ambientes','estrenar','cochera']) 

In [ ]:
#Armamos nuestran matriz de features (X) y nuestro vector objetivo (y):
feature_cols = ['surface_total_in_m2','surface_covered_in_m2','ambientes','estrenar','cochera']
X = df_properatti[feature_cols]
y = df_properatti.ARS_to_USD_corregido
    
#Separamos los datos en sets de testeo y training:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
    
#Instanciamos el modelo:
linreg = linear_model.LinearRegression()
    
#Entrenamos el modelo:
reg_lineal_mult = linreg.fit(X_train, y_train)
    
#Calculamos el y_pred:
y_pred = linreg.predict(X_test)
    
#Guardamos en una variable al Score (R2) del modelo para poder comparar:
R2_reg_lineal_mult = reg_lineal_mult.score(X_test, y_test)
    
print (feature_cols)
#Imprimimos los coeficientes:
print('Intercept: ', linreg.intercept_.round(3))
print('Coeficients: ',linreg.coef_.round(3))
#para observarlo mejor miramos el nombre de la variable con el coeficiente:
print(list(zip(feature_cols, linreg.coef_.round(3))))
    
#Imprimimos todas las métricas que nos interesan para poder comparar:
print ('y_test_sample:' , y_test.values[0:10])
print ('y_pred_sample:', y_pred[0:10].astype(int))
print ('MAE: ', metrics.mean_absolute_error(y_test, y_pred).round(3))
print ('MSE: ', metrics.mean_squared_error(y_test, y_pred).round(3))
print ('RMSE: ', np.sqrt(metrics.mean_squared_error(y_test, y_pred)).round(3))
print ('R2_train: ', reg_lineal_mult.score(X_train, y_train).round(3))
print ('R2_test: ', reg_lineal_mult.score(X_test, y_test).round(3))
print()

In [ ]:
#Creamos un diccionario con la informacion para luego amar un dataframe y comparar los modelos:

reg_multiple_sin_reg_model = {'Modelo': "Regresion Lineal Múltiple sin Regularización",
                        'Alcance del modelo': 'Buenos Aires',
                        'Variable Objetivo': 'ARS_to_USD_corregido',
                        'Cantidad de Observaciones': len(df_properatti),
                        'R2_train': reg_lineal_mult.score(X_train, y_train),
                        'R2_test': reg_lineal_mult.score(X_test, y_test),
                        'Intercepto': reg_lineal_mult.intercept_}

In [ ]:
plt.plot(y,y, '-.',c='grey')
plt.scatter(y_pred, y_test, s=30, c='r', marker='+', zorder=10)
plt.xlabel("ARS_to_USD_predict")
plt.ylabel("ARS_to_USD_real")
plt.title('Comparación entre y_pred y los valores y reales', pad = 30)
plt.show()

#### Conclusión de la Regresión Lineal Multiple aplicada a las features de manera grupal
Ahora podemos ver como la tendencia de las predicciones se acercan más a la linea diagonal (que es la recta a la cual tiene que tender las predicciones) por lo cual podemos decir que el modelo mejoró con respecto a las regresiones lineales simples, pero aún se puede ajustar para una predicción más precisa.

In [ ]:
plt.scatter(df_properatti.surface_total_in_m2, y, s=30, c='purple', marker='+', zorder=10, alpha = 0.5, label = "m2 total")
plt.scatter(df_properatti.surface_covered_in_m2, y, s=30, c='black', marker='.', zorder=10, alpha = 0.5, label = "m2 cubierto")
plt.scatter(df_properatti.ambientes, y, s=10, c='green', marker='o', zorder=10, alpha = 0.5, label = "ambientes")
plt.xlabel("Valores estandarizados")
plt.ylabel("Precio en USD")
plt.title('Valores estandarizados')
plt.legend(loc="center", bbox_to_anchor=(0.5, -.20), shadow=False, ncol=2)
plt.show()

plt.plot(y,y, '-.',c='grey')
plt.scatter(y_pred, y_test, s=30, c='r', marker='+', zorder=10)
plt.xlabel("Precio en USD predichos")
plt.ylabel("Precio en USD")
plt.title('y_real vs y_pred')
plt.show()

<a id="section_5.2"></a> 
## 5.2 Regresión Lineal Multiple _CON Regularización_ 
(con estandarizacion de variables numericas ni variables dummies para variables categoricas)

[volver a TOC](#section_TOC)

### Estandarizacion de features numericas:
Primero vamos a Normalizar (Estandarizar) las Features Numéricas: Con esto llevamos a las variables numéricas a la misma escala (desv estandar = 1 y media = 0) y le damos el mismo peso a todas. 
Las variables dummies no es necesario que sean normalizadas ya que son variables dicotomicas (0 o 1). 
Recordar que en la lista numericals vamos a poner a todas las variables numéricas, pero no a la variable Target (precio).

In [ ]:
# Importamos las librerías de Regularización:
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model

In [ ]:
#agregaba 2 varables cuadráticas para ver si el modelo mejoraba, pero no lo hizo..
# df_properatti['ambientes_2'] = df_properatti.ambientes * df_properatti.ambientes
# df_properatti['sup_total_2'] = df_properatti.surface_total_in_m2 * df_properatti.surface_total_in_m2

In [ ]:
#usamos StandarScaler como para escaloar los valores ya que MaxMin puede causar problemas con los outliers que puedan haber
numericals = ['surface_total_in_m2','surface_covered_in_m2', 'ambientes']#,'sup_total_2', 'ambientes_2']

X = df_properatti[numericals]
scaler = StandardScaler() 
X_std = scaler.fit_transform(X)    
std_df = pd.DataFrame(X_std)
std_df.columns = [i + '_std' for i in numericals] 
std_df.head(3)

In [ ]:
#Chequeamos que esté normalizado el std_df:
std_df.describe().round(5)

In [ ]:
#ploteamos los valores estandarizados vs los no estandarizados
plt.scatter(df_properatti.surface_total_in_m2, y, s=30, c='purple', marker='+', zorder=10, alpha = 0.5, label = "m2 total")
plt.scatter(df_properatti.surface_covered_in_m2, y, s=30, c='black', marker='.', zorder=10, alpha = 0.5, label = "m2 cubierto")
plt.scatter(df_properatti.ambientes, y, s=10, c='green', marker='o', zorder=10, alpha = 0.5, label = "ambientes")
plt.xlabel("Valores NO estandarizados")
plt.ylabel("Precio en USD")
plt.title('Valores NO estandarizados')
plt.legend(loc="center", bbox_to_anchor=(1.2, .50), shadow=False)
plt.show()

plt.scatter(std_df.surface_total_in_m2_std, y, s=30, c='purple', marker='+', zorder=10, alpha = 0.5, label = "m2 total")
plt.scatter(std_df.surface_covered_in_m2_std, y, s=30, c='black', marker='.', zorder=10, alpha = 0.5, label = "m2 cubierto")
plt.scatter(std_df.ambientes_std, y, s=10, c='green', marker='o', zorder=10, alpha = 0.5, label = "ambientes")
plt.xlabel("Valores estandarizados")
plt.ylabel("Precio en USD")
plt.title('Valores estandarizados')
plt.legend(loc="center", bbox_to_anchor=(1.2, .50), shadow=False)
plt.show()

plt.plot(y,y, '-.',c='grey')
plt.scatter(y_pred, y_test, s=30, c='r', marker='+', zorder=10)
plt.xlabel("Precio en USD predichos")
plt.ylabel("Precio en USD")
plt.title('y_real vs y_pred')
plt.show()

Podemos ver como tiene efecto la estandarización de los datos y podemos tener una mejor visión de los datos dentro de un mismo gráfico.

<br>Vamos a crear el Dummy Dataframe, en el que vamos a seleccionar solamente a las columnas dummies que habíamos creado antes en el Dataframe "df_properatti". Recordemos que las variables dummies las habíamos concatenado a nuestro dataframe original, y que empezaban en la columna 13 del Dataframe:

In [ ]:
dummies_df = df_properatti.iloc[:,13:]#262] si es que uso las variables cuadráticas creadas más arriba
dummies_df.head(3)

Entonces X, que es el dataset de features que usamos para el entrenamiento, va a ser concatenar todas las variables dummies que construimos antes (dummies_df) con todas las variables numéricas estandarizadas (std_df) más las variables Estrenar y Cochera. 
Este X es básicamente nuestro nuevo dataset que vamos a usar para entrenar el modelo de Regresión Lineal

In [ ]:
#valores que usaremos de ahora en adelante como X e y
X = pd.concat([dummies_df, std_df,df_properatti[['estrenar', 'cochera']].astype(int)], axis=1)
y = df_properatti.ARS_to_USD_corregido

X.tail(3)

## Stats model

Trabajamos sobre con Stats Model para conocer una descripción profunda de los coeficientes con los que estamos trabajando en el modelo

In [ ]:
import statsmodels.api as sm

X_const = sm.add_constant(X)

model = sm.OLS(y, X_const).fit()

mod_summary = model.summary()
mod_param = model.params
mod_pvalues = np.round(model.pvalues,3)

In [ ]:
mod_summary

<a id="section_lineal_ridge"></a> 
## 6. Regresión Lineal Ridge

[volver a TOC](#section_TOC)

Ahora trabajaremos en las regularizaciones, para ello seteamos ciertos parametros que se aplicarán por igual a todos los modelos que usaremos.

Creamos la primera regularización, usamos la de Ridge. 
Guardamos todos los resultados obtenidos para poder compararlos al final con todos los modelos aplicados y poder definir el modelo que más se adapte a nuestros datos

In [ ]:
#Separamos el conjunto en train y test:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

#seteamos los valores que usaremos para los cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=12)

In [ ]:
#Instanciamos el modelo de Ridge:

lm_ridgeCV = linear_model.RidgeCV(alphas=[0.0001, 0.0005, 0.001, 0.005, 0.01,\
                                        0.05, 0.1, 1, 5, 10, 100], cv=kf) 
#Entrenamos el modelo
reg_lineal_ridgeCV = lm_ridgeCV.fit(X_train, y_train)

#Guardamos el score en una variable:
score_ridgeCV = reg_lineal_ridgeCV.score(X_train, y_train)

#Prediccion con el set de testeo:
print('Score del modelo Ridge:', score_ridgeCV)

In [ ]:
# Veamos cuál es el alpha óptimo de Ridge:
best_alpha_ridgeCV = lm_ridgeCV.alpha_
best_alpha_ridgeCV

In [ ]:
model_ridge = linear_model.Ridge(alpha = best_alpha_ridgeCV, fit_intercept = True, normalize = False)
model_fit_ridge = model_ridge.fit(X_train, y_train)
print('intercepto_rd :', model_fit_ridge.intercept_.round(3))
print('coeficientes_rd :', model_fit_ridge.coef_.round(3))
print('score_rd :', model_fit_ridge.score(X_test, y_test))

In [ ]:
(model_fit_ridge.coef_ == 0).sum()

In [ ]:
#Creamos un diccionario para luego mostrar los datos en un dataframe:
ridge_model = {'Modelo': "Regresión Lineal Ridge",
                          'Alcance del modelo': 'Buenos Aires',
                          'Variable Objetivo': 'ARS_to_USD_corregido',
                          'Cantidad de Observaciones': len(df_properatti),
                          'R2_train': score_ridgeCV,
                          'R2_test': model_fit_ridge.score(X_test, y_test),
                          'Intercepto': reg_lineal_ridgeCV.intercept_,
                          'Alpha del modelo': best_alpha_ridgeCV}

### Analizamos como reacciona R² y alpha a los cambios en el % de test_size y en el n de folds, en la regularización de Ridge

Queremos conocer los movimientos de R2 tanto de train como de test cuando alteramos los valores del porcentaje del test set sobre el train set

In [ ]:
#Analizamos los cambios de R2 y alpha cuando variamos el % de test usado para testear el modelo

cv_check_test_size = []
for i in np.linspace (0.05, 0.95, num=19):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=42)
    kf = KFold(n_splits=5, shuffle=True, random_state=12)
    lm_ridgeCV = linear_model.RidgeCV(alphas=[0.0001, 0.0005, 0.001, 0.005, 0.01,\
                                        0.05, 0.1, 1, 1.5, 2, 2.5, 3, 4, 5, 10, 100], cv=kf) 
    reg_lineal_ridgeCV = lm_ridgeCV.fit(X_train, y_train)
    score_ridgeCV_train = reg_lineal_ridgeCV.score(X_train, y_train)
    score_ridgeCV_test = reg_lineal_ridgeCV.score(X_test, y_test)
    best_alpha_ridgeCV = lm_ridgeCV.alpha_

    cv_check_test_size_1 = (i.round(2), score_ridgeCV_train, score_ridgeCV_test,best_alpha_ridgeCV)
    cv_check_test_size.append(cv_check_test_size_1)

df_array_test_size = pd. DataFrame(cv_check_test_size, columns = ['test_size','R2_train','R2_test','best_alpha'])

df_array_test_size

In [ ]:
#Graficos de los R2
plt.plot(df_array_test_size.test_size, df_array_test_size.R2_train, c='r', label='R2_train'),
plt.plot(df_array_test_size.test_size, df_array_test_size.R2_test, c='g', label='R2_test'),
plt.xlabel('Test Size')
plt.ylabel('R2 values')
plt.legend(loc = 'lower left')
plt.show()

#grafico de alpha
plt.plot(df_array_test_size.test_size, df_array_test_size.best_alpha, c='b', label = 'alpha'),
plt.xlabel('Test Size'),
plt.ylabel('Alpha values'),
plt.legend(loc = 'lower right')
plt.show()

Ahora queremos conocer los movimientos de R2 tanto de train como de test cuando alteramos los números de folds que usamos al trabajar con la validación cruzada

In [ ]:
#Analizamos los cambios de R2 y alpha cuando variamos el n de folds (no tiene cambios, eso esta correcto?)

cv_check_folds = []
for i in range (2, 11):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)
    kf = KFold(n_splits=i, shuffle=True, random_state=12)
    lm_ridgeCV = linear_model.RidgeCV(alphas=[0.0001, 0.0005, 0.001, 0.005, 0.01,\
                                        0.05, 0.1, 1, 5, 10, 100], cv=kf) 
    reg_lineal_ridgeCV = lm_ridgeCV.fit(X_train, y_train)
    score_ridgeCV_train = reg_lineal_ridgeCV.score(X_train, y_train)
    score_ridgeCV_test = reg_lineal_ridgeCV.score(X_test, y_test)
    best_alpha_ridgeCV = lm_ridgeCV.alpha_

    cv_check_folds_1 = (i, score_ridgeCV_train, score_ridgeCV_test,best_alpha_ridgeCV)
    cv_check_folds.append(cv_check_folds_1)

df_array_folds = pd. DataFrame(cv_check_folds, columns = ['Folds','R2_train','R2_test','best_alpha'])
df_array_folds

In [ ]:
#graficos de los R2
plt.plot(df_array_folds.Folds, df_array_folds.R2_train, c='r', label='R2_train'),
plt.plot(df_array_folds.Folds, df_array_folds.R2_test, c='g', label='R2_test'),
plt.xlabel('Folds')
plt.ylabel('R2 values')
plt.legend()
plt.show()

#grafico de alpha
plt.plot(df_array_folds.Folds, df_array_folds.best_alpha, c='b', label = 'alpha'),
plt.xlabel('Folds'),
plt.ylabel('Alpha values'),
plt.legend()
plt.show()

<a id="section_lineal_lasso"></a> 
## 7. Regresión Lineal Lasso

[volver a TOC](#section_TOC)

In [ ]:
import warnings

warnings.filterwarnings('ignore')

In [ ]:
#Instanciamos el modelo de Lasso:
lm_lassoCV =  linear_model.LassoCV(alphas=np.linspace(0.001, 100, num = 100), cv = kf, max_iter = 20000, tol=0.000001)

#Entrenamos el modelo
reg_lineal_lassoCV = lm_lassoCV.fit(X_train, y_train)

#Guardamos el score
score_lassoCV = reg_lineal_lassoCV.score(X_train, y_train)

#Prediccion con el set de testeo:
print('Score del modelo Lasso:', score_lassoCV)

In [ ]:
# Veamos cuál es el alpha óptimo de Lasso:
best_alpha_lassoCV = lm_lassoCV.alpha_
best_alpha_lassoCV

In [ ]:
model_lasso = linear_model.Lasso(alpha = best_alpha_lassoCV, fit_intercept = True, max_iter = 20000)
model_fit_lasso = model_lasso.fit(X_train, y_train)
print('intercepto_l :', model_fit_lasso.intercept_.round(3))
print('coeficientes_l :', model_fit_lasso.coef_.round(3))
print('score_l :', model_fit_lasso.score(X_test, y_test))

In [ ]:
#Creamos un diccionario para luego mostrar los datos en un dataframe:
lasso_model = {'Modelo': "Regresión Lineal Lasso",
                          'Alcance del modelo': 'Buenos Aires',
                          'Variable Objetivo': 'ARS_to_USD_corregido',
                          'Cantidad de Observaciones': len(df_properatti),
                          'R2_train': score_lassoCV,
                          'R2_test': model_fit_lasso.score(X_test, y_test),
                          'Intercepto': reg_lineal_lassoCV.intercept_,
                          'Alpha del modelo': best_alpha_lassoCV}

#### Chequeo Georeferencial
Chequeamos geográficamente si hay relación entre los coeficientes de lasso = 0 y los p-value de los coeficientes de StatsModel (SM) mayores a 0.005

In [ ]:
#Eliminamos la constante de los parámetros de sm, para comparar con los otros coeficientes:
mod_pvalues = mod_pvalues.drop(mod_pvalues.index[0])

In [ ]:
dictio={'Col_X': X.columns,
       'lasso_coef': model_fit_lasso.coef_,
       'sm_pvalue': mod_pvalues,
       'ridge_coef': model_fit_ridge.coef_}

In [ ]:
data_params = pd.DataFrame (dictio)
data_params.head()

In [ ]:
#Coinciden los lasso = 0 con los p_values > 0.005? No, pero resulta útil visualizarlo gráficamente:
p_v = 0.005

data_params["coincidencias"] = (data_params.lasso_coef == 0) & (data_params.sm_pvalue > p_v)


print(f'Coef Lasso = 0:     {(data_params.lasso_coef == 0).sum()}  {np.round((data_params.lasso_coef == 0).sum()/data_params.shape[0]*100, 2)}%')
print(f'P_values > 0.005:  {(data_params.sm_pvalue > p_v).sum()}  {np.round((data_params.sm_pvalue > p_v).sum()/data_params.shape[0]*100, 2)}%')
print('total de datos:   ',   data_params.shape[0])
print()
print(f'{data_params["coincidencias"].value_counts()}')

In [ ]:
#Data_params.to_csv(path_or_buf='analisis.csv', sep=';', encoding = 'latin1', decimal= ',')

In [ ]:
#Este código es para ver sólo un valor por place involucrado en los coef. analizados.
geo_properatti_2 = geo_properatti.drop_duplicates (subset = 'place_name')
geo_properatti_params = data_params.merge(geo_properatti_2, left_on = 'Col_X', right_on='place_name', how = 'left')
geo_properatti_params.sample(3)

In [ ]:
#Este código es para ver todos los valores del barrio involucrado, en los coef. analizados:
# geo_properatti_params = data_params.merge(geo_properatti, left_on = 'Col_X', right_on='place_name', how = 'left')
# geo_properatti_params.sample(3)

In [ ]:
geo_properatti_sm = geo_properatti_params[geo_properatti_params.sm_pvalue > p_v]
geo_properatti_lasso = geo_properatti_params[geo_properatti_params.lasso_coef == 0]

In [ ]:
geo_properatti_sm = gpd.GeoDataFrame(geo_properatti_sm)
geo_properatti_lasso = gpd.GeoDataFrame(geo_properatti_lasso)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
departamentos_arg.plot(ax=ax, color='white', edgecolor='black')


geo_properatti_sm.plot(color='red', ax=ax, zorder=5, markersize = 50, marker = 'x', label = 'p_value > 0.005')
geo_properatti_lasso.plot(color='green', ax=ax, zorder=5, markersize = 20, marker = 'o', label = 'Lasso coef = 0')


ax.set_title('Relacion geografica entre coeficientes con inconsistencias', 
             pad = 20, 
             fontdict={'fontsize':20, 'color': '#4873ab'})

ax.set_xlabel('Longitud')
ax.set_ylabel('Latitud')

ax.set(xlim=(-59.1, -58.3), ylim=( -34.95, -34.25))

ax.legend()

fig.show()

<a id="section_elastic_net"></a> 
## 8. Regresión Elastic Net

[volver a TOC](#section_TOC)

In [ ]:
#Instanciamos el modelo:
lm_elastic_netCV =  linear_model.ElasticNetCV(l1_ratio= [.1, .5, .7, .9, .95, .99, 1], alphas = [0.0001, 0.001, 0.01, 1.0, 5, 10, 20, 50, 100, 1000], cv = kf)

#Entrenamos el modelo:
elastic_netCV = lm_elastic_netCV.fit(X_train, y_train)

#Guardamos el score:
score_EN_CV = elastic_netCV.score(X_train, y_train)

#Predicción con el set de testeo:
print('Score del modelo Elastic Net:', score_EN_CV)

In [ ]:
# Veamos cuál es el alpha óptimo de Elastic Net:
best_alpha_EN_CV = lm_elastic_netCV.alpha_
best_alpha_EN_CV

In [ ]:
model_EN = linear_model.ElasticNet(l1_ratio= 0.5, alpha = best_alpha_EN_CV, fit_intercept = True, normalize = False)
model_fit_EN = model_EN.fit(X_train, y_train)
print(model_fit_EN.intercept_)
print(model_fit_EN.coef_)
print(model_fit_EN.score(X_test, y_test))

In [ ]:
#Creamos un diccionario para luego mostrar los datos en un dataframe:

Elastic_Net_model = {'Modelo': "Elastic Net",
                        'Alcance del modelo': 'Buenos Aires',
                        'Variable Objetivo': 'ARS_to_USD_corregido',
                        'Cantidad de Observaciones': len(df_properatti),
                        'R2_train': score_EN_CV,
                        'R2_test': model_fit_EN.score(X_test, y_test),
                        'Intercepto': elastic_netCV.intercept_,
                        'Alpha del modelo': elastic_netCV.alpha_,
                        'l1_ratio': elastic_netCV.l1_ratio_}


<a id="section_conclusiones"></a> 
## 9. Conclusiones

[volver a TOC](#section_TOC)

Para poder presentar las conclusiones de nuestro análisis, lo primero que haremos es reunir los resultados de cada modelo, guardados en diccionarios por separado, para tener todo en un único Dataframe y compararlos más cómodamente.

<a id="section_9.1"></a> 
### 9.1 Comparación Numérica de los Distintos Modelos

[volver a TOC](#section_TOC)

In [ ]:
# Creamos una lista de todos los diccionarios:
diccionarios = [
reg_lineal_simple_model_m2,
reg_lineal_simple_model_amb,
reg_lineal_simple_model_est,
reg_lineal_simple_model_cochera,
reg_lineal_simple_model_scm2,
reg_multiple_sin_reg_model,
ridge_model,                
lasso_model,
Elastic_Net_model
]

In [ ]:
#Generamos el Dataframe con toda la info:
resultados = pd.DataFrame(diccionarios)

#Ordenamos las columnas en el siguiente orden para verlo mas comodamente:
resultados = resultados[['Modelo','Alcance del modelo','Variable Objetivo','Cantidad de Observaciones','R2_train','R2_test','Intercepto','Alpha del modelo','l1_ratio']]

In [ ]:
#Chequeamos que se haya generado correctamente:
resultados
#Faltaría una línea mas en el Dataframe con los resultados de la Regresión Lineal Múltiple sin Estandarización de variables y sin variables Dummies,
# que lo agregaré cuando arreglemos lo de la columna 'ambientes'. En ese modelo usamos solo a "surface_total_in_m2" y a "ambientes" como
# variables predictoras:

In [ ]:
#Pasamos los datos a un archivo para guardarlo:

#resultados.to_csv(path_or_buf='Resultados.csv')

<br>
A continuación mostraremos en un Groupby y una Pivot Table cuál es el mejor modelo basado en el R2:
<br>

In [ ]:
#Si lo queremos ver como Groupby:
resultados_R2= resultados.groupby('Modelo').mean()['R2_train'].sort_values(ascending=False).round(4)
resultados_R2

In [ ]:
#Si lo queremos ver como Pivot Table:
resultados.pivot_table(['R2_train','R2_test'],['Modelo'],aggfunc='mean').round(4).sort_values(by='R2_test',ascending=False)

<a id="section_9.2"></a> 
### 9.2 Comparación Gráfica de los Distintos Modelos

[volver a TOC](#section_TOC)

A continuación mostraremos en un gráfico de Barras cuál es el mejor modelo, basado en el R2:

In [ ]:
# Hacemos el gráfico:

y = np.log(resultados_R2)
my_cmap = plt.get_cmap('bwr') #buscar distintos colores en: https://matplotlib.org/stable/tutorials/colors/colormaps.html#sphx-glr-tutorials-colors-colormaps-py
rescale = lambda y: (y - np.min(y)) / (np.max(y) - np.min(y))

colors = ['black', 'maroon', 'darkgreen', 'navy', 'grey', 'orangered', 'limegreen', 'royalblue']
chart = resultados_R2.sort_values().plot.barh(figsize=(14,5),fontsize=15, color = my_cmap(rescale(y)))
chart.set_title("R2 según Tipo de Modelo", fontsize=25)
chart.set_ylabel("Modelo", fontsize=25)
chart.set_xlabel("R2", fontsize=25)
plt.show()

Podemos arribar a la conclusión de que los modelos que mejor R² obtuvieron fueron los de Regresión Lineal Ridge, Regresión Lineal Lasso y Elastic Net, de mayor a menor R², respectivamente.

#### Los coeficientes que más influencia tienen en la Regresión lineal son:

In [ ]:
data_params.lasso_coef = abs(data_params.lasso_coef)
Coef_importantes = data_params.sort_values('lasso_coef',ascending=False).iloc[0:41]

y = np.log(Coef_importantes.lasso_coef)
my_cmap = plt.get_cmap('Wistia')
rescale = lambda y: (y - np.min(y)) / (np.max(y) - np.min(y))

plt.figure(figsize=(20,4))
plt.bar(Coef_importantes.Col_X, Coef_importantes.lasso_coef, color = my_cmap(rescale(y)))
plt.xticks(Coef_importantes.Col_X, Coef_importantes.Col_X, rotation=90)
plt.title("Los 40 coeficientes más influyentes en el modelo Lasso", fontsize=25)

plt.show()

In [ ]:
data_params.lasso_coef = abs(data_params.lasso_coef)
Coef_importantes = data_params.sort_values('lasso_coef',ascending=True).iloc[0:96]

y = (Coef_importantes.lasso_coef)
my_cmap = plt.get_cmap('winter')
rescale = lambda y: (y - np.min(y)) / (np.max(y) - np.min(y))

plt.figure(figsize=(23,4))
plt.bar(Coef_importantes.Col_X, Coef_importantes.lasso_coef, color = my_cmap(rescale(y)))
plt.xticks(Coef_importantes.Col_X, Coef_importantes.Col_X, rotation=90)
plt.title("Los 95 coeficientes menos influyentes en el modelo Lasso", fontsize=25)

plt.show()

In [ ]:
data_params.ridge_coef = abs(data_params.ridge_coef)
Coef_importantes = data_params.sort_values('ridge_coef',ascending=False).iloc[0:41]

y = np.log(Coef_importantes.ridge_coef)
my_cmap = plt.get_cmap('Wistia')
rescale = lambda y: (y - np.min(y)) / (np.max(y) - np.min(y))

plt.figure(figsize=(20,4))
plt.bar(Coef_importantes.Col_X, Coef_importantes.ridge_coef, color = my_cmap(rescale(y)))
plt.xticks(Coef_importantes.Col_X, Coef_importantes.Col_X, rotation=90)
plt.title("Los 40 coeficientes más influyentes en el modelo Ridge", fontsize=25)

plt.show()

In [ ]:
data_params.ridge_coef = abs(data_params.ridge_coef)
Coef_importantes = data_params.sort_values('ridge_coef',ascending=True).iloc[0:41]

y = (Coef_importantes.ridge_coef)
my_cmap = plt.get_cmap('winter')
rescale = lambda y: (y - np.min(y)) / (np.max(y) - np.min(y))

plt.figure(figsize=(20,4))
plt.bar(Coef_importantes.Col_X, Coef_importantes.ridge_coef, color = my_cmap(rescale(y)))
plt.xticks(Coef_importantes.Col_X, Coef_importantes.Col_X, rotation=90)
plt.title("Los 40 coeficientes menos influyentes en el modelo Ridge", fontsize=25)

plt.show()